In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, *-*"


#!import config/Settings.cs
#!import plugins/FilesPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();

__Agents__

The following code creates the agents.

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

var ProgamManager = """
    You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the 
    user requirements and form the detail documents with requirements and costing. 
""";

ChatCompletionAgent ProgaramManagerAgent =
            new()
            {
                Instructions = ProgamManager,
                Name = nameof(ProgaramManagerAgent),
                Kernel = kernel
            };

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

var ProductOwner = """
   You are Product Owner, and your goal is to write proper Product Backlog Items. 
   You take all the requirements fgiven by the Program Manager and write proper Backlog Items.
   The PBI has a title, description and a list of Accecteance Criteria.
   
   The description sould follow the following schema:
   As a <Role | User, Developer etc > I want to .....
""";

ChatCompletionAgent ProductOwnerAgent =
            new()
            {
                Instructions = ProductOwner,
                Name = nameof(ProductOwnerAgent),
                Kernel = kernel
            };

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

string ScrumMaster = """
    You are manager which will review ProductOwner Product Backlog Items, and make sure all client requirements are in.
    Once the Product Backlog Item is in a good shape you can approve the request by just responding "approve"
""";

 ChatCompletionAgent ScrumMasterAgent =
            new()
            {
                Instructions = ScrumMaster,
                Name = nameof(ScrumMasterAgent),
                Kernel = kernel
            };

In [ ]:
using System.Threading;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;

#pragma warning disable SKEXP0110, SKEXP0001 
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
}

In [ ]:
using Microsoft.SemanticKernel.Agents;
#pragma warning disable SKEXP0110, SKEXP0001 

AgentGroupChat chat =
            new(ProgaramManagerAgent, ProductOwnerAgent, ScrumMasterAgent)
            {
                ExecutionSettings =
                    new()
                    {
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                Agents = [ScrumMasterAgent],
                                MaximumIterations = 6,
                            }
                    }
            };

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

var input = """
I want to develop a calculator application.
The calculator shall be used to do scientific calculations.
Beside of basic artihmertics also sin functions, log, and root and pow functions should be posible.
Please help me.
""";

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}